<a href="https://colab.research.google.com/github/leman-cap13/my_projects/blob/main/TMNIST_high_low_level_API_Torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#High Level API

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import zipfile

In [ ]:
!kaggle datasets download nimishmagre/tmnist-typeface-mnist

In [ ]:
with zipfile.ZipFile('/content/tmnist-typeface-mnist.zip','r') as zip_ref:
  zip_ref.extractall()

In [ ]:
import pandas as pd
df=pd.read_csv('/content/TMNIST_Data.csv')
df

In [ ]:
# it's just flattened for CSV storage.

In [ ]:
print(df.shape)

In [ ]:
fonts = df.iloc[:, 0].values
fonts

In [ ]:
import numpy as np
y = df.iloc[:, 1].values.astype(np.int32)
y

In [ ]:
# Extract and normalize pixel values
X = df.iloc[:, 2:].values.astype(np.float32) / 255.0

the first element represents the font name (ex-Chivo-Italic, Sen-Bold), the second element represents the label (a number from 0-9) and the remaining 784 elements represent the grayscale pixel values (from 0-255) for the 28x28 pixel image.

In [ ]:
X_images=X.reshape(-1,28,28,1)

-1 means calculate n_samples by yourself 28x28 shape of image and 1 indicates channel. if it is grayscale we need to use 1 , on the other hand if it is 3 means we are using RGB (red,green,blue) channel

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for i in range(20):
  plt.subplot(4,5,i+1)
  plt.xticks()
  plt.yticks()
  plt.grid(False)
  plt.imshow(X_images[i])
  plt.xlabel(fonts[i])

In [ ]:
data_path='/content/TMNIST_Data.csv'

In [ ]:
import torch
import torch.nn as nn

torch.nn is equivalent to --> import tensorflow as tf

In [ ]:
model=nn.Sequential(
    nn.Conv2d(in_channels=1,out_channels=32,kernel_size=5,stride=1,padding=1),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=2,padding=1),
    nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=2,padding=1),
    nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,stride=1,padding=1),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(in_features=512,out_features=128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(in_features=128,out_features=64),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(in_features=64,out_features=32),
    nn.Linear(in_features=32, out_features=10)

)

PyTorch nn.Linear and nn.Conv2d don’t accept a kernel_initializer argument directly

in tensorflow when we make model we use ([]), differently in pytorch we only use() not extra[]

nn.MaxPool2d(kernel_size=5) should take kernel_size argument as distinc from tensorflow

nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0)
Conv2d doest have filters agrument as tensor has

in PyTorch we cannot directly specify activation functions or kernel initializers inside nn.Linear

nn.Linear(in_features, out_features)

PyTorch: layers as separate positional args inside ()

Keras: layers inside a list [] passed as a single argument inside ()

In [ ]:
loss=nn.CrossEntropyLoss()


In [ ]:
import torch.optim as optim
optimization=optim.Adam(model.parameters(), lr=0.001)

** in PyTorch, there is no .compile() step**

need to define three key components manually:
1. Loss function
2. Optimizer
3. Metrics

In [ ]:
!pip install torchkeras


In [ ]:
from torchkeras import KerasModel
final_model=KerasModel(model,loss_fn=loss,optimizer=optimization,metrics=['accuracy'])


PyTorch doesn’t track metrics automatically. we need to calculate them manually during training:

torchkeras is a wrapper built on top of PyTorch.

It borrows the syntax and convenience of Keras (e.g., compile(), fit(), evaluate()), but under the hood, it still runs pure PyTorch code

 still writing a PyTorch model, but using a Keras-like interface to simplify training and evaluation

In [ ]:
#DataLoader for train and val dataset
from torch.utils.data import DataLoader,TensorDataset
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
#first convert data to tensors

X_tensor=torch.tensor(X_images).permute(0,3,1,2)
y_tensor=torch.tensor(y).long()

# Original X_images is a NumPy array of shape:[batch, height, width, channels] ,
#but PyTorch expects images in this shape:[batch,channels, height, width],
#need to convert from [N, H, W, C] → [N, C, H, W]
#permute(0, 3, 1, 2) it does dimension rearrangement
#nn.CrossEntropyLoss expects class labels as integers that represent class indices (e.g., 0,1,2,...).
#These indices must be integers, but not just any integer — specifically int64 (long) in PyTorch.
#torch.long ≡ torch.int64     int64 aka long
X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)


#Now split train test
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X_tensor,y_tensor,test_size=0.2, random_state=42)


#Convert TensorDataset
train_data=TensorDataset(X_train,y_train)
val_data=TensorDataset(X_val,y_val)

#Create DataLoader
train_loader=DataLoader(train_data,batch_size=32,shuffle=True)
val_loader=DataLoader(val_data,batch_size=32,shuffle=False)


Even though you're using a high-level API, PyTorch still expects DataLoader objects for train_data and val_data

Purpose of shuffling: Prevent the model from learning the order of the data.
Shuffling introduces randomness, helping the model generalize better and reducing overfitting.
Validation is for measurement, not learning

In [ ]:
#Train the model
final_model.fit(train_loader,val_data=val_loader, epochs=5)

In [ ]:
final_model.evaluate(val_loader)

In [ ]:
sample = X_val[0].unsqueeze(0)
logits = final_model.net(sample)
prediction = torch.argmax(logits, dim=1)

print("Predicted label:", prediction.item())


tensor.unsqueeze(dim) returns a new tensor with a dimension of size 1 inserted at the specified position.
unsqueeze() is dumb — it just inserts a dimension of size 1 at the index you specify.
It doesn’t know about:
batches
channels
images

x = torch.randn(3, 28, 28)   # shape: [3, 28, 28]
x.unsqueeze(0).shape         # → [1, 3, 28, 28]
x.unsqueeze(1).shape         # → [3, 1, 28, 28]
x.unsqueeze(2).shape         # → [3, 28, 1, 28]

In [ ]:
import matplotlib.pyplot as plt
import torch

# Select a sample image and label
sample = X_val[7]                      # Shape: [1, 28, 28]
label = y_val[7].item()

# Add batch dimension for model input
input_tensor = sample.unsqueeze(0)    # Shape: [1, 1, 28, 28]

# Run through the model
logits = final_model.net(input_tensor)
prediction = torch.argmax(logits, dim=1).item()

# Plot the image
plt.imshow(sample.cpu().squeeze(0).numpy(), cmap='gray')
plt.title(f"Predicted: {prediction} | True: {label}")
plt.axis('off')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 4, figsize=(12, 9))
fig.suptitle("Model Predictions on Validation Samples", fontsize=16)

for i in range(12):
    sample = X_val[i]
    label = y_val[i].item()
    input_tensor = sample.unsqueeze(0)
    logits = final_model.net(input_tensor)
    prediction = torch.argmax(logits, dim=1).item()

    ax = axes[i // 4, i % 4]
    ax.imshow(sample.squeeze(0).cpu().numpy(), cmap='gray')
    ax.set_title(f"Pred: {prediction} | True: {label}")
    ax.axis('off')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
plt.figure(figsize=(10,10))
for i in range(12):
    sample = X_val[i]
    label = y_val[i].item()
    input_tensor = sample.unsqueeze(0)
    logits = final_model.net(input_tensor)
    prediction = torch.argmax(logits, dim=1).item()

    plt.subplot(3,4,i+1)
    plt.grid(False)
    plt.imshow(X_val[i].squeeze(0).cpu().numpy(), cmap='gray')
    plt.xlabel(f"Pred: {prediction} ")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_true = []
y_pred = []
for X_batch, y_batch in val_loader:
    logits = final_model.net(X_batch)
    preds = torch.argmax(logits, dim=1)
    y_true.extend(y_batch.cpu().numpy())
    y_pred.extend(preds.cpu().numpy())

cm = confusion_matrix(y_true, y_pred)
ConfusionMatrixDisplay(cm).plot()


#Low Level API

In [ ]:
data_path='/content/TMNIST_Data.csv'

In [ ]:
import numpy as np

In [ ]:
import pandas as pd
data=pd.read_csv(data_path)
data.head()

In [ ]:
data=data.drop('names',axis=1)

In [ ]:
labels=data['labels']
images=data.drop('labels',axis=1)

In [ ]:
labels

In [ ]:
images

In [ ]:
# first of all i need to normalize and reshape the images in order to work on it
images=images.astype(np.float32) / 255.0
images=images.values.reshape(-1,1,28,28)


In [ ]:
#but we need tensor so ve convert it to tensor data
images=torch.tensor(images)
labels=torch.tensor(labels.values, dtype=torch.long)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for i in range(12):
  plt.subplot(3,4,i+1)
  plt.grid(False)
  plt.imshow(images[i].squeeze(0), cmap='gray')
  plt.xlabel(labels[i].item())


In [ ]:
#now split train validation
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(images,labels,test_size=0.2, random_state=42, stratify=labels)

In [ ]:
#not i need to make train_data and validation_data to sent datacollator
train_data=torch.utils.data.TensorDataset(X_train,y_train)
val_data=torch.utils.data.TensorDataset(X_val,y_val)


A Data Collator is a function or class that takes a list of examples (from a dataset) and turns them into a batch

A DataLoader is a tool that fetches your data in mini-batches so that you can train your model efficiently and in parallel.

In [ ]:
# Now use Dataloader
import torch
from torch.utils.data import DataLoader,TensorDataset
train_loader=DataLoader(train_data,batch_size=32,shuffle=True)
val_loader=DataLoader(val_data,batch_size=32,shuffle=False)


In [ ]:
# Make Convulation model
import torch.nn as nn

torch.manual_seed(42)

conv1=nn.Conv2d(in_channels=1,out_channels=32,kernel_size=5,stride=1,padding=1)
batch_norm1=nn.BatchNorm2d(32)
max_pool1=nn.MaxPool2d(kernel_size=2)

conv2=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=2,padding=1)
batch_norm2=nn.BatchNorm2d(64)
conv3=nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1)
batch_norm3=nn.BatchNorm2d(128)
max_pool2=nn.MaxPool2d(kernel_size=2)


conv4=nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=2,padding=1)
batch_norm4 = nn.BatchNorm2d(256)
conv5=nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,stride=1,padding=1)
batch_norm5 = nn.BatchNorm2d(512)
max_pool3=nn.MaxPool2d(kernel_size=2)

flatten=nn.Flatten()
linear1=nn.Linear(in_features=512,out_features=64)
dropout=nn.Dropout(0.3)
linear2=nn.Linear(in_features=64,out_features=32)
linear3=nn.Linear(in_features=32,out_features=10)


in_features for linear1 must be channels * height * width.

In [ ]:
#need the forward pass function that defines how the input flows through these layers.
import torch.nn.functional as F

def forward(x):
  x=conv1(x)
  x=batch_norm1(x)
  x=F.relu(x)
  x=max_pool1(x)

  x=conv2(x)
  x=batch_norm2(x)
  x=F.relu(x)

  x=conv3(x)
  x=batch_norm3(x)
  x=F.relu(x)
  x=max_pool2(x)

  x=conv4(x)
  x=batch_norm4(x)
  x=F.relu(x)

  x=conv5(x)
  x=batch_norm5(x)
  x=F.relu(x)
  x=max_pool3(x)

  x=flatten(x)
  x=linear1(x)
  x=F.relu(x)
  x=dropout(x)
  x=linear2(x)
  x=F.relu(x)
  x=dropout(x)
  x=linear3(x)

  return x


In [ ]:
x_dummy = torch.randn(4, 1, 28, 28)  # batch of 4 images
out = forward(x_dummy)
print(out.shape)  # should be [4, 10] because we have 10 classes and 4 batch size


In [ ]:
#  Now we need to set up our training essentials loss optimizer and metrics

loss_fn=nn.CrossEntropyLoss()

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class MyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, stride=1, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.max_pool1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(32, 64, 3, stride=2, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.batch_norm3 = nn.BatchNorm2d(128)
        self.max_pool2 = nn.MaxPool2d(2)

        self.conv4 = nn.Conv2d(128, 256, 3, stride=2, padding=1)
        self.batch_norm4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, 3, stride=1, padding=1)
        self.batch_norm5 = nn.BatchNorm2d(512)
        self.max_pool3 = nn.MaxPool2d(2)

        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(512, 64)
        self.dropout = nn.Dropout(0.3)
        self.linear2 = nn.Linear(64, 32)
        self.linear3 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.max_pool1(x)

        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.max_pool2(x)

        x = self.conv4(x)
        x = self.batch_norm4(x)
        x = F.relu(x)

        x = self.conv5(x)
        x = self.batch_norm5(x)
        x = F.relu(x)
        x = self.max_pool3(x)

        x = self.flatten(x)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.linear3(x)
        return x


In [ ]:
my_model=MyCNN()

In [ ]:
#optimizer
optimizer=torch.optim.Adam(my_model.parameters(),lr=0.001)

In [ ]:
# set Accelerate
from accelerate import Accelerator
accelerator=Accelerator()

my_model,optimizer,train_loader,val_loader=accelerator.prepare(my_model,optimizer,train_loader,val_loader)

In [ ]:
from transformers import get_scheduler
num_epochs=5
num_training_steps=len(train_loader)*num_epochs


lr_scheduler=get_scheduler(
    name='linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
#Custom training loop
from tqdm.auto import tqdm

progress_bar=tqdm(range(num_training_steps))

my_model.train()
num_epochs=5

for epoch in range(num_epochs):
  my_model.train()
  for batch in train_loader:
    inputs,labels=batch
    inputs = inputs.float()
    outputs=my_model(inputs)
    loss=loss_fn(outputs,labels)
    accelerator.backward(loss)

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)


   #evaluation data's turn
  my_model.eval()
  all_preds=[]
  all_labels=[]
  for batch in val_loader:
    with torch.no_grad():
      inputs,labels=batch
      inputs = inputs.float()
      outputs=my_model(inputs)
      preds=torch.argmax(outputs,dim=1)

      preds=accelerator.gather(preds)
      labels=accelerator.gather(labels)

      all_preds.append(preds)
      all_labels.append(labels)

  all_preds = torch.cat(all_preds).cpu()
  all_labels = torch.cat(all_labels).cpu()






In [ ]:
def postprocess(all_preds, all_labels):
    preds = all_preds.cpu().numpy()
    labels = all_labels.cpu().numpy()
    return preds, labels


preds, labels = postprocess(all_preds, all_labels)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(labels, preds)
print(f"Epoch {epoch+1}/{num_epochs} | Val Accuracy: {accuracy:.4f}")



In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

accuracy = accuracy_score(labels, preds)
f1 = f1_score(labels, preds, average='macro')

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")

print("Classification Report:")
print(classification_report(labels, preds))


In [ ]:
import matplotlib.pyplot as plt
import torch


my_model.eval()
device = next(my_model.parameters()).device

plt.figure(figsize=(12, 9))
num_to_show = 12
count = 0

with torch.no_grad():
    for batch in val_loader:
        inputs, labels = batch
        inputs = inputs.float().to(device)
        labels = labels.to(device)

        outputs = my_model(inputs)
        preds = torch.argmax(outputs, dim=1)

        batch_size = inputs.size(0)
        for i in range(batch_size):
            if count >= num_to_show:
                break

            image = inputs[i].squeeze(0).cpu().numpy()
            pred_label = preds[i].cpu().item()
            true_label = labels[i].cpu().item()

            plt.subplot(3, 4, count + 1)
            plt.imshow(image, cmap='gray')
            plt.title(f"Pred: {pred_label} | True: {true_label}")
            plt.axis('off')

            count += 1

        if count >= num_to_show:
            break

plt.tight_layout()
plt.show()
